## **FTL: Focusing on local nodes**

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Flatten
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


Extracted preprocessing for https://github.com/polvalls9/Transfer-Learning-Based-Intrusion-Detection-in-5G-and-IoT-Networks.git

In [ ]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['label']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['label']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [ ]:
bot = models.load_model('./w_bot_5_5.hdf5')

In [ ]:
bot.trainable = False

In [ ]:
bot.pop()
bot.pop()
bot.pop()

In [ ]:
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']

### Imbalanced 3

#### Node 1

In [ ]:
node = pd.read_csv('../imb3_part1.csv')

In [ ]:
train, val = train_test_split(node, test_size=0.2, random_state=42, shuffle = True)

In [ ]:
xtrain, ytrain = preprocessing(train)
xval, yval = preprocessing(val)

In [ ]:
model = models.Sequential()
model.add(bot)
model.add(Flatten())

model.add(Dense(units=128, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(rate=0.4))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(rate=0.4))
model.add(Dense(units=192, activation='relu'))
model.add(Dropout(rate=0.3))

model.add(Dense(units=2, activation='softmax'))

filepath = '../node0w_ftl_imb3.hdf5'
callbacks = [
        keras.callbacks.EarlyStopping(
            monitor = 'val_loss', # Use accuracy to monitor the model
            patience = 2 # Stop after 10 steps with lower accuracy
        ),
        keras.callbacks.ModelCheckpoint(
            filepath = filepath, # file where the checkpoint is saved
            monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
            save_best_only = True)]# Only save model if it is the best
optimizer = keras.optimizers.Adam(learning_rate=9e-4)
model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
history = model.fit(xtrain, ytrain, epochs=20, validation_data=(xval, yval), callbacks=callbacks, batch_size=2048)
model.save(filepath)

print(model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy'])


#### Node 2

In [ ]:
node = pd.read_csv('../imb3_part2.csv')

In [ ]:
train, val = train_test_split(node, test_size=0.2, random_state=42, shuffle = True)

In [ ]:
xtrain, ytrain = preprocessing(train)
xval, yval = preprocessing(val)

In [ ]:
model = models.Sequential()
model.add(bot)
model.add(Flatten())

model.add(Dense(units=448, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units=224, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.0))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(rate=0.3))


model.add(Dense(units=2, activation='softmax'))

filepath = '../node1w_ftl_imb3.hdf5'
callbacks = [
        keras.callbacks.EarlyStopping(
            monitor = 'val_loss', # Use accuracy to monitor the model
            patience = 2 # Stop after 10 steps with lower accuracy
        ),
        keras.callbacks.ModelCheckpoint(
            filepath = filepath, # file where the checkpoint is saved
            monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
            save_best_only = True)]# Only save model if it is the best
optimizer = keras.optimizers.Adam(learning_rate=3e-3)
model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
history = model.fit(xtrain, ytrain, epochs=20, validation_data=(xval, yval), callbacks=callbacks, batch_size=2048)
model.save(filepath)

print(model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy'])



#### Node 3

In [ ]:
node = pd.read_csv('../imb3_part3.csv')

In [ ]:
train, val = train_test_split(node, test_size=0.2, random_state=42, shuffle = True)

In [ ]:
xtrain, ytrain = preprocessing(train)
xval, yval = preprocessing(val)

In [ ]:
model = models.Sequential()
model.add(bot)
model.add(Flatten())

model.add(Dense(units=224, activation='relu'))
model.add(Dropout(rate=0.1))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(rate=0.1))
model.add(Dense(units=256, activation='relu'))
model.add(Dropout(rate=0.1))


model.add(Dense(units=2, activation='softmax'))

filepath = '../node2w_ftl_imb3.hdf5'
callbacks = [
        keras.callbacks.EarlyStopping(
            monitor = 'val_loss', # Use accuracy to monitor the model
            patience = 2 # Stop after 10 steps with lower accuracy
        ),
        keras.callbacks.ModelCheckpoint(
            filepath = filepath, # file where the checkpoint is saved
            monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
            save_best_only = True)]# Only save model if it is the best
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
history = model.fit(xtrain, ytrain, epochs=20, validation_data=(xval, yval), callbacks=callbacks, batch_size=2048)
model.save(filepath)

print(model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy'])



#### Node 4

In [ ]:
node = pd.read_csv('../imb3_part4.csv')

In [ ]:
train, val = train_test_split(node, test_size=0.2, random_state=42, shuffle = True)

In [ ]:
xtrain, ytrain = preprocessing(train)
xval, yval = preprocessing(val)

In [ ]:
model = models.Sequential()
model.add(bot)
model.add(Flatten())

#model.add(Dense(units=448, activation='relu'))
#model.add(Dropout(rate=0.3))
#model.add(Dense(units=224, activation='relu'))
#model.add(Dropout(rate=0.3))
#model.add(Dense(units=64, activation='relu'))
#model.add(Dropout(rate=0.0))
#model.add(Dense(units=256, activation='relu'))
#model.add(Dropout(rate=0.3))

model.add(Dense(units=480, activation='relu'))
model.add(Dropout(rate=0.4))
model.add(Dense(units=160, activation='relu'))
model.add(Dropout(rate=0.1))
model.add(Dense(units=480, activation='relu'))
model.add(Dropout(rate=0.0))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(rate=0.4))

model.add(Dense(units=2, activation='softmax'))

filepath = '/node3w_ftl_imb3.hdf5'
callbacks = [
        keras.callbacks.EarlyStopping(
            monitor = 'val_loss', # Use accuracy to monitor the model
            patience = 2 # Stop after 10 steps with lower accuracy
        ),
        keras.callbacks.ModelCheckpoint(
            filepath = filepath, # file where the checkpoint is saved
            monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
            save_best_only = True)]# Only save model if it is the best
optimizer = keras.optimizers.Adam(learning_rate=3e-3)
model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
history = model.fit(xtrain, ytrain, epochs=20, validation_data=(xval, yval), callbacks=callbacks, batch_size=2048)
model.save(filepath)

print(model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy'])



#### Node 5

In [ ]:
node = pd.read_csv('../imb3_part5.csv')

In [ ]:
train, val = train_test_split(node, test_size=0.2, random_state=42, shuffle = True)

In [ ]:
xtrain, ytrain = preprocessing(train)
xval, yval = preprocessing(val)

In [ ]:
model = models.Sequential()
model.add(bot)
model.add(Flatten())

model.add(Dense(units=192, activation='relu'))
model.add(Dropout(rate=0.1))
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(units=384, activation='relu'))
model.add(Dropout(rate=0.1))
model.add(Dense(units=480, activation='relu'))
model.add(Dropout(rate=0.2))



model.add(Dense(units=2, activation='softmax'))

filepath = '../node4w_ftl_imb3.hdf5'
callbacks = [
        keras.callbacks.EarlyStopping(
            monitor = 'val_loss', # Use accuracy to monitor the model
            patience = 2 # Stop after 10 steps with lower accuracy
        ),
        keras.callbacks.ModelCheckpoint(
            filepath = filepath, # file where the checkpoint is saved
            monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
            save_best_only = True)]# Only save model if it is the best
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
history = model.fit(xtrain, ytrain, epochs=20, validation_data=(xval, yval), callbacks=callbacks, batch_size=2048)
model.save(filepath)

print(model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy'])

